# Equipment Templates

1. Run the [Initializations](#Initializations)

Whereas equipment classes in Haystack and Brick are pretty general (ahu, Air_Handling_Unit), the purpose of an Equipment Template is to define very concrete expectations of the expected properties and point types. It might not designate itself completely as a 'new class type', however, it is more akin to providing a configuration for a class.  Equipment templates utilize a similar structure to a PGT.

##  Template Structure Keys:
### Required
- `id`: (str) a UUID4.  Globally unique.
- `symbol`: (str) A shorthand representation.
- `template_type`: (str) For an Equipment Template, this will always be `equipment-template`.
- `schema_name`: (str) The base schema this template is based on.  Currently supported are 'Brick' or 'Haystack'. TODO: Add GDB.
- `version`: (str) The version of the schema this is based on.
- `extends`: (str) The 'base equipment' class that this template is providing a configuration for.

### Optional
- `description`: description
- `properties`: TODO
- `telemetry_point_type`: These can be defined as typical EntityTemplate, or can refer to a PointGroupTemplate `symbol`.
    - PGT: To denote a set of `telemetry_point_type`s by using a PGT `symbol`, the value should be `None`.  The `symbol` will be found based on the set of `{schema_name, version, symbol}` (pulls in `schema_name` and `version`) from the Equipment Template.
    - EntityTemplate: follows the EntityTemplate definition syntax
    
Below we initialize two Equipment Template dictionaries

In [3]:
haystack_eqt_dict = {
    'id': 'b6f175ef-e172-4a02-9aa6-8d930a8222a8',
    'symbol': 'VAV_CO_SD',
    'description': 'Single duct, cooling only VAV',
    'template_type': 'equipment-template',
    'schema_name': 'Haystack',
    'version': '3.9.9',
    'extends': 'coolingOnly-vav',
    'properties': {
        'singleDuct': {'_kind': 'marker'},
        'ratedAirflow': {'_kind': 'number', 'val': 123}
    },
    'telemetry_point_types': {
        'SD': None,
        'damper-cmd-point': {'curVal': {'_kind': 'bool'}}
    }
}

brick_eqt_dict = {
    'id': 'b6b75737-c33b-4c20-88a2-0f9f91271af9',
    'symbol': 'VAV_CO_SD',
    'description': 'Single duct, cooling only VAV',
    'template_type': 'equipment-template',
    'schema_name': 'Brick',
    'version': '1.1',
    'extends': 'Variable_Air_Volume_Box',
    'properties': {},
    'telemetry_point_types': {
        'SD': None,
        'Damper_Position_Command': {}
    }
}

## Populating Equipment Templates

We initialize the Equipment Templates and check that they are valid (i.e. that all required keys have been defined).

In [4]:
heqt = tt.EquipmentTemplate(**haystack_eqt_dict)
beqt = tt.EquipmentTemplate(**brick_eqt_dict)

In [5]:
print(heqt.is_valid)
print(beqt.is_valid)

True
True


We then:
1. populate the template basics
1. resolve the `extends`.

This ensures that this template is extending an equipment from the expected schema.  The query we use to check are:
- Haystack: `"SELECT ?e WHERE {{ ?e rdfs:subClassOf* phIoT:equip }}"`
- Brick: `"SELECT ?e WHERE {{ ?e rdfs:subClassOf* brick:Equipment }}"`

In [6]:
heqt.populate_template_basics()
beqt.populate_template_basics()
heqt.resolve_extends()
beqt.resolve_extends()

[tasty.templates.hget_entity_classes] Permutation time: 0.00 seconds


Once the extension has resolved, we then attempt to resolve the telemetry points.  This happens in two steps:
1. Where an EntityTemplate is defined, this is created or resolves to an existing EntityTemplate.
1. Where a `symbol` is expected to resolve to a PGT, the PGT must first be populated in order to correctly resolve.  We will see this in the following.

In [7]:
heqt.resolve_telemetry_point_types()

# This Equipment Template has not fully resolved.
heqt.fully_resolved

[tasty.templates.EquipmentTemplate] No PointGroupTemplate found for Haystack, 3.9.9, SD. Make sure PGTs have been loaded
[tasty.templates.hget_entity_classes] Permutation time: 0.01 seconds
tasty.templates.EntityTemplate created and is valid: Haystack, 3.9.9, {'point'}


False

We then populate the necessary PGT in our workspace, which allows us then to successfully resolve.

In [8]:
hpgt = tt.PointGroupTemplate(**haystack_pgt_dict)
hpgt.populate_template_basics()
hpgt.resolve_telemetry_point_types()

heqt.resolve_telemetry_point_types()

# Our Equipment Template should now fully resolve
heqt.fully_resolved

[tasty.templates.hget_entity_classes] Permutation time: 6.40 seconds
tasty.templates.EntityTemplate created and is valid: Haystack, 3.9.9, {'his-point', 'cur-point'}
[tasty.templates.hget_entity_classes] Permutation time: 0.15 seconds
tasty.templates.EntityTemplate created and is valid: Haystack, 3.9.9, {'point'}
[tasty.templates.hget_entity_classes] Permutation time: 0.01 seconds
tasty.templates.EntityTemplate created and is valid: Haystack, 3.9.9, {'point'}
[tasty.templates.EquipmentTemplate] Found PointGroupTemplate with id: 4aa753fc-ab1b-47d0-984f-121fa0cfa0e9
[tasty.templates.hget_entity_classes] Permutation time: 0.01 seconds
tasty.templates.EntityTemplate Equivalent EntityTemplate already exists and was returned.
tasty.templates.EntityTemplate created and is valid: Haystack, 3.9.9, {'point'}
[tasty.templates.EquipmentTemplate] Fully Resolved


True

## Getting All Entity Templates

At the end of the day, PointGroupTemplates are mainly a grouping of a set of points for reusability.  EquipmentTemplates that implement a PGT easily resolve to the actual EntityTemplates.

In [14]:
all_ets_used = heqt.get_all_points_as_entity_templates()

for et in all_ets_used:
    print(f"Simple typing: {et.get_simple_typing_info()} Simple properties: {et.get_simple_properties()}")

Simple typing: {'his-point', 'discharge', 'cur-point', 'air', 'sensor', 'flow'} Simple properties: {'curVal': {'_kind': 'number', 'val': None}, 'unit': {'_kind': 'str', 'val': 'cfm'}}
Simple typing: {'point', 'discharge', 'sp', 'air', 'flow'} Simple properties: {}
Simple typing: {'damper', 'point', 'cmd'} Simple properties: {'unit': {'val': 'percent'}}
Simple typing: {'damper', 'point', 'cmd'} Simple properties: {'curVal': {'_kind': 'bool'}}


# Initializations

In [2]:
# imports and setup
import os
import uuid

import tasty.templates as tt
import tasty.constants as tc
import tasty.graphs as tg

from loaders import initialize_eqt_notebook

haystack_pgt_dict = initialize_eqt_notebook()

[tasty.templates.hget_entity_classes] Permutation time: 6.43 seconds
tasty.templates.EntityTemplate Equivalent EntityTemplate already exists and was returned.
tasty.templates.EntityTemplate created and is valid: Brick, 1.1, {'Damper_Position_Command'}
tasty.templates.EntityTemplate Equivalent EntityTemplate already exists and was returned.
tasty.templates.EntityTemplate created and is valid: Haystack, 3.9.9, {'motor', 'writable-point', 'cur-point'}
